# Thiết lập Môi trường và tải dữ liệu

In [1]:
import tarfile

tar_path = r"E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\hwu.tar"
extract_to = r"E:\Nam4\NLP\level 1\NLP_22001295_HUS\data"

with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_to)

print("Giải nén thành công!")


Giải nén thành công!


In [2]:
import pandas as pd

# Dữ liệu có thể được phân tách bằng tab và không có header
df_train = pd.read_csv(r'E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\hwu\train.csv', names=['text', 'intent'], header=0)
df_val = pd.read_csv(r'E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\hwu\val.csv', names=['text', 'intent'], header=0)
df_test = pd.read_csv(r'E:\Nam4\NLP\level 1\NLP_22001295_HUS\data\hwu\test.csv', names=['text', 'intent'], header=0)

print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)
print("Test shape:", df_test.shape)

df_train.head(100)

Train shape: (8954, 2)
Validation shape: (1076, 2)
Test shape: (1076, 2)


,text,intent
0,what alarms do i have set right now,alarm_query
1,checkout today alarm of meeting,alarm_query
2,report alarm settings,alarm_query
3,see see for me the alarms that you have set to...,alarm_query
4,is there an alarm for ten am,alarm_query
...,...,...
95,check if i have set any alarm for morning,alarm_query
96,list all of my upcoming alarms,alarm_query
97,my alarms,alarm_query
98,do i have any alarms set for six am tomorrow,alarm_query


In [3]:
# Tiền xử lý nhãn 
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

all_intents = pd.concat([df_train['intent'], df_val['intent'], df_test['intent']], axis=0)
label_encoder.fit(all_intents)

df_train['label'] = label_encoder.transform(df_train['intent'])
df_val['label'] = label_encoder.transform(df_val['intent'])
df_test['label'] = label_encoder.transform(df_test['intent'])

num_classes = len(label_encoder.classes_)
print(f"Số lượng intent (classes): {num_classes}")
print(f"Các intent: {label_encoder.classes_[:10]}...") 

df_train.head()

Số lượng intent (classes): 64
Các intent: ['alarm_query' 'alarm_remove' 'alarm_set' 'audio_volume_down'
 'audio_volume_mute' 'audio_volume_up' 'calendar_query' 'calendar_remove'
 'calendar_set' 'cooking_recipe']...


,text,intent,label
0,what alarms do i have set right now,alarm_query,0
1,checkout today alarm of meeting,alarm_query,0
2,report alarm settings,alarm_query,0
3,see see for me the alarms that you have set to...,alarm_query,0
4,is there an alarm for ten am,alarm_query,0


# 1. Pipeline TF-IDF + Logistic Regression

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

# 1. Tạo một pipeline với TfidfVectorizer và LogisticRegression
tfidf_lr_pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression(max_iter=1000, random_state=42)
)

# 2. Huấn luyện pipeline trên tập train
print("Đang huấn luyện TF-IDF + Logistic Regression...")
tfidf_lr_pipeline.fit(df_train['text'], df_train['label'])
print("Huấn luyện hoàn tất!")

# 3. Đánh giá trên tập validation
print("\n=== Đánh giá trên tập Validation ===")
y_val_pred = tfidf_lr_pipeline.predict(df_val['text'])
val_accuracy = accuracy_score(df_val['label'], y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print("\nClassification Report (Validation):")
print(classification_report(df_val['label'], y_val_pred, target_names=label_encoder.classes_))


# 4. Đánh giá trên tập test
print("Đánh giá trên tập Test")
y_test_pred = tfidf_lr_pipeline.predict(df_test['text'])
test_accuracy = accuracy_score(df_test['label'], y_test_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")
print("\nClassification Report (Test):")
print(classification_report(df_test['label'], y_test_pred, target_names=label_encoder.classes_))

Đang huấn luyện TF-IDF + Logistic Regression...
Huấn luyện hoàn tất!

=== Đánh giá trên tập Validation ===
Validation Accuracy: 0.8587

Classification Report (Validation):
                          precision    recall  f1-score   support

             alarm_query       0.78      0.95      0.86        19
            alarm_remove       0.75      0.55      0.63        11
               alarm_set       0.74      0.74      0.74        19
       audio_volume_down       0.62      0.62      0.62         8
       audio_volume_mute       1.00      0.53      0.70        15
         audio_volume_up       0.79      0.85      0.81        13
          calendar_query       0.77      0.53      0.62        19
         calendar_remove       0.82      0.95      0.88        19
            calendar_set       0.88      0.79      0.83        19
          cooking_recipe       0.94      0.89      0.92        19
        datetime_convert       0.56      0.62      0.59         8
          datetime_query       0.82

e:\Users\Miniconda\envs\nlp-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\Users\Miniconda\envs\nlp-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
e:\Users\Miniconda\envs\nlp-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


# 2. Pipeline Word2Vec (Trung bình) + DenseLayer

In [62]:
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping



# 1. Huấn luyện mô hình Word2Vec trên dữ liệu text
print("Đang huấn luyện mô hình Word2Vec...")
sentences = [text.split() for text in df_train['text']]
w2v_model = Word2Vec(sentences, vector_size=128, window=8, min_count=1, workers=5, epochs=150)
print("Huấn luyện Word2Vec hoàn tất!")

# 2. Viết hàm để chuyển mỗi câu thành vector trung bình
def sentence_to_avg_vector(text, model):
    """Chuyển một câu thành vector trung bình của các từ"""
    words = text.split()
    # Lấy vector của các từ có trong vocabulary
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:
        # Nếu không có từ nào trong vocabulary, trả về vector 0
        return np.zeros(model.vector_size)
    
    # Trả về vector trung bình
    avg_vector = np.mean(word_vectors, axis=0)
    return avg_vector

# 3. Tạo dữ liệu train/val/test X_train_avg, X_val_avg, X_test_avg
print("Đang chuyển đổi văn bản thành vector...")
X_train_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in df_train['text']])
X_val_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in df_val['text']])
X_test_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in df_test['text']])

# Chuyển label sang dạng one-hot encoding
y_train_cat = to_categorical(df_train['label'], num_classes=num_classes)
y_val_cat = to_categorical(df_val['label'], num_classes=num_classes)
y_test_cat = to_categorical(df_test['label'], num_classes=num_classes)

print(f"X_train_avg shape: {X_train_avg.shape}")
print(f"X_val_avg shape: {X_val_avg.shape}")
print(f"X_test_avg shape: {X_test_avg.shape}")

# 4. Xây dựng mô hình Sequential của Keras
model_w2v = Sequential([
    Dense(128, activation='relu', input_shape=(w2v_model.vector_size,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# 5. Compile mô hình
model_w2v.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTóm tắt mô hình:")
model_w2v.summary()

# 6. Huấn luyện mô hình
print("\nĐang huấn luyện mô hình Word2Vec + DenseLayer...")
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history = model_w2v.fit(
    X_train_avg, y_train_cat,
    validation_data=(X_val_avg, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


# 7. Đánh giá trên tập test
print("\nĐánh giá trên tập Test")
test_loss, test_accuracy = model_w2v.evaluate(X_test_avg, y_test_cat, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Dự đoán và tạo classification report
y_test_pred = model_w2v.predict(X_test_avg)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

print("\nClassification Report (Test):")
print(classification_report(df_test['label'], y_test_pred_classes, target_names=label_encoder.classes_))

Đang huấn luyện mô hình Word2Vec...
Huấn luyện Word2Vec hoàn tất!
Đang chuyển đổi văn bản thành vector...
X_train_avg shape: (8954, 128)
X_val_avg shape: (1076, 128)
X_test_avg shape: (1076, 128)

Tóm tắt mô hình:


e:\Users\Miniconda\envs\nlp-env\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_64 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 64)             │         4,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,928 (113.00 KB)

 Trainable params: 28,928 (113.00 KB)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện mô hình Word2Vec + DenseLayer...
Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2042 - loss: 3.2875 - val_accuracy: 0.6097 - val_loss: 1.7789
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4496 - loss: 1.9958 - val_accuracy: 0.6952 - val_loss: 1.1838
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5461 - loss: 1.6105 - val_accuracy: 0.7203 - val_loss: 0.9886
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5833 - loss: 1.4347 - val_accuracy: 0.7621 - val_loss: 0.8938
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6215 - loss: 1.3239 - val_accuracy: 0.7667 - val_loss: 0.8414
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6356 - loss: 1.2521 - val_accuracy: 0.7835 - val_loss: 0.8095
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6632 - loss: 1.1714 - val_accuracy: 0.7825 - val_loss: 0.7886
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

# 3. Mô hình Nâng cao (Embedding Pre-trained + LSTM)

In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional, LayerNormalization


# 1. Tiền xử lý cho mô hình chuỗi
print("Tiền xử lý dữ liệu cho mô hình LSTM")

# a. Tokenizer: Tạo vocab và chuyển text thành chuỗi chỉ số
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(df_train['text'])

# Chuyển text thành sequences
train_sequences = tokenizer.texts_to_sequences(df_train['text'])
val_sequences = tokenizer.texts_to_sequences(df_val['text'])
test_sequences = tokenizer.texts_to_sequences(df_test['text'])

# b. Padding: Đảm bảo các chuỗi có cùng độ dài
max_len = 50
X_train_pad = pad_sequences(train_sequences, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(val_sequences, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(test_sequences, maxlen=max_len, padding='post')

print(f"X_train_pad shape: {X_train_pad.shape}")
print(f"X_val_pad shape: {X_val_pad.shape}")
print(f"X_test_pad shape: {X_test_pad.shape}")

# 2. Tạo ma trận trọng số cho Embedding Layer từ Word2Vec
print("\nTạo ma trận Embedding từ Word2Vec")
# đọc file GloVe
glove_path = r'E:\Nam4\NLP\glove.6B.50d.txt\glove.6B.50d.txt'
embedding_index = {}
with open(glove_path, encoding='utf8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

# Tạo ma trận embedding
embedding_dim = 50 
vocab_size_actual = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size_actual, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"Số từ có embedding trong GloVe: {np.sum(np.any(embedding_matrix != 0, axis=1))}/{vocab_size_actual}")
print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"Số từ có trong Word2Vec: {np.sum(np.any(embedding_matrix != 0, axis=1))}/{vocab_size_actual}")

Tiền xử lý dữ liệu cho mô hình LSTM
X_train_pad shape: (8954, 50)
X_val_pad shape: (1076, 50)
X_test_pad shape: (1076, 50)

Tạo ma trận Embedding từ Word2Vec
Embedding matrix shape: (4265, 50)
Số từ có embedding trong GloVe: 3967/4265
Embedding matrix shape: (4265, 50)
Số từ có trong Word2Vec: 3967/4265


In [68]:
# Tham số chung
lstm_units = 128
dense_units = 64
dropout_lstm = 0.2
recurrent_dropout_lstm = 0.2
dropout_dense = 0.3

In [69]:
# 3. Xây dựng mô hình Sequential với LSTM
print("\nXây dựng mô hình LSTM với Pre-trained Embedding")
lstm_model_pretrained = Sequential([
    Embedding(
        input_dim=vocab_size_actual,
        output_dim=embedding_dim,         # embedding pre-trained
        weights=[embedding_matrix],       # khởi tạo từ Word2Vec/GloVe
        trainable=True,                   # fine-tune embedding
        input_length=max_len
    ),
    Bidirectional(LSTM(lstm_units, dropout=dropout_lstm, recurrent_dropout=recurrent_dropout_lstm, return_sequences=False)),
    Dense(dense_units, activation='relu'),
    Dropout(dropout_dense),
    Dense(num_classes, activation='softmax')
])


# Compile mô hình
lstm_model_pretrained.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTóm tắt mô hình:")
lstm_model_pretrained.build(input_shape=(None, max_len))
lstm_model_pretrained.summary()

# 4. Huấn luyện với EarlyStopping
print("\nHuấn luyện mô hình LSTM")
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history_lstm = lstm_model_pretrained.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_val_pad, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# 5. Đánh giá trên tập test
print("\nĐánh giá trên tập Test")
test_loss_pretrained, test_accuracy_pretrained = lstm_model_pretrained.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"Test Loss: {test_loss_pretrained:.4f}")
print(f"Test Accuracy: {test_accuracy_pretrained:.4f}")

# Dự đoán và tạo classification report
y_test_pred_lstm = lstm_model_pretrained.predict(X_test_pad)
y_test_pred_lstm_classes = np.argmax(y_test_pred_lstm, axis=1)

print("\nClassification Report (Test):")
print(classification_report(df_test['label'], y_test_pred_lstm_classes, target_names=label_encoder.classes_))


Xây dựng mô hình LSTM với Pre-trained Embedding

Tóm tắt mô hình:


Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)        │ (None, 50, 50)         │       213,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_19                │ (None, 256)            │       183,296 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 64)             │         4,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 417,154 (1.59 MB)

 Trainable params: 417,154 (1.59 MB)

 Non-trainable params: 0 (0.00 B)


Huấn luyện mô hình LSTM
Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 28s 74ms/step - accuracy: 0.1698 - loss: 3.3462 - val_accuracy: 0.5195 - val_loss: 1.9281
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.4567 - loss: 1.8908 - val_accuracy: 0.6952 - val_loss: 1.1621
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 18s 65ms/step - accuracy: 0.6071 - loss: 1.3563 - val_accuracy: 0.7388 - val_loss: 0.8951
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.6919 - loss: 1.0363 - val_accuracy: 0.7928 - val_loss: 0.7166
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - accuracy: 0.7508 - loss: 0.8535 - val_accuracy: 0.8188 - val_loss: 0.6495
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.7933 - loss: 0.7159 - val_accuracy: 0.8225 - val_loss: 0.5844
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.8202 - loss: 0.6169 - val_accuracy: 0.8411 - val_loss: 0.5533
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 19s 66ms/step - accuracy:

# 4. Mô hình Nâng cao (Embedding học từ đầu + LSTM)

In [65]:
# Dữ liệu đã được tiền xử lý (tokenized, padded) từ nhiệm vụ 3
# Sử dụng lại X_train_pad, X_val_pad, X_test_pad và y_train_cat, y_val_cat, y_test_cat

# 1. Xây dựng mô hình LSTM với Embedding học từ đầu
print("Xây dựng mô hình LSTM với Embedding học từ đầu")

lstm_model_scratch = Sequential([
    Embedding(
        input_dim=vocab_size_actual,
        output_dim=100,      # embedding học từ đầu
        trainable=True,
        input_length=max_len
    ),
    Bidirectional(LSTM(lstm_units, dropout=dropout_lstm, recurrent_dropout=recurrent_dropout_lstm, return_sequences=False)),
    Dense(dense_units, activation='relu'),
    Dropout(dropout_dense),
    Dense(num_classes, activation='softmax')
])

# 2. Compile mô hình
lstm_model_scratch.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTóm tắt mô hình:")
lstm_model_scratch.build(input_shape=(None, max_len))
lstm_model_scratch.summary()

# 3. Huấn luyện với EarlyStopping
print("\nHuấn luyện mô hình LSTM (Embedding học từ đầu)")

early_stopping_scratch = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history_lstm_scratch = lstm_model_scratch.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_val_pad, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping_scratch],
    verbose=1
)

# 4. Đánh giá trên tập test
print("\nĐánh giá trên tập Test")
test_loss_scratch, test_accuracy_scratch = lstm_model_scratch.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"Test Loss: {test_loss_scratch:.4f}")
print(f"Test Accuracy: {test_accuracy_scratch:.4f}")

# Dự đoán và tạo classification report
y_test_pred_scratch = lstm_model_scratch.predict(X_test_pad)
y_test_pred_scratch_classes = np.argmax(y_test_pred_scratch, axis=1)

print("\nClassification Report (Test):")
print(classification_report(df_test['label'], y_test_pred_scratch_classes, target_names=label_encoder.classes_))


Xây dựng mô hình LSTM với Embedding học từ đầu

Tóm tắt mô hình:


e:\Users\Miniconda\envs\nlp-env\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 50, 100)        │       426,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, 256)            │       234,496 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 64)             │         4,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 681,604 (2.60 MB)

 Trainable params: 681,604 (2.60 MB)

 Non-trainable params: 0 (0.00 B)


Huấn luyện mô hình LSTM (Embedding học từ đầu)
Epoch 1/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 25s 71ms/step - accuracy: 0.0933 - loss: 3.6853 - val_accuracy: 0.3931 - val_loss: 2.3187
Epoch 2/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.4451 - loss: 1.9489 - val_accuracy: 0.6933 - val_loss: 1.1798
Epoch 3/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 80ms/step - accuracy: 0.6670 - loss: 1.1623 - val_accuracy: 0.7853 - val_loss: 0.8286
Epoch 4/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.7802 - loss: 0.7859 - val_accuracy: 0.8104 - val_loss: 0.7400
Epoch 5/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - accuracy: 0.8396 - loss: 0.5670 - val_accuracy: 0.8262 - val_loss: 0.6797
Epoch 6/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step - accuracy: 0.8794 - loss: 0.4291 - val_accuracy: 0.8309 - val_loss: 0.6757
Epoch 7/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 22s 77ms/step - accuracy: 0.9073 - loss: 0.3344 - val_accuracy: 0.8355 - val_loss: 0.6887
Epoch 8/50
280/280 ━━━━━━━━━━━━━━━━━━━━ 20

# 5. Đánh giá, so sánh, phân tích

In [70]:
import pandas as pd
from sklearn.metrics import f1_score

print("PHẦN 1: SO SÁNH ĐỊNH LƯỢNG CÁC MÔ HÌNH")

# Thu thập kết quả từ các mô hình

# Model 1: TF-IDF + Logistic Regression
y_pred_tfidf = tfidf_lr_pipeline.predict(df_test['text'])
f1_tfidf = f1_score(df_test['label'], y_pred_tfidf, average='macro')
loss_tfidf = 'N/A'

# Model 2: Word2Vec (Avg) + Dense
y_pred_w2v = np.argmax(model_w2v.predict(X_test_avg), axis=1)
f1_w2v = f1_score(df_test['label'], y_pred_w2v, average='macro')
loss_w2v = test_loss

# Model 3: Embedding (Pre-trained) + LSTM
y_pred_lstm_pretrained = np.argmax(lstm_model_pretrained.predict(X_test_pad), axis=1)
f1_lstm_pretrained = f1_score(df_test['label'], y_pred_lstm_pretrained, average='macro')
loss_lstm_pretrained = test_loss_pretrained

# Model 4: Embedding (Scratch) + LSTM
y_pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(X_test_pad), axis=1)
f1_lstm_scratch = f1_score(df_test['label'], y_pred_lstm_scratch, average='macro')
loss_lstm_scratch = test_loss_scratch

# Tạo bảng so sánh
results_df = pd.DataFrame({
    'Pipeline': [
        'TF-IDF + Logistic Regression',
        'Word2Vec (Avg) + Dense',
        'Embedding (Pre-trained) + LSTM',
        'Embedding (Scratch) + LSTM'
    ],
    'F1-score (Macro)': [
        f'{f1_tfidf:.4f}',
        f'{f1_w2v:.4f}',
        f'{f1_lstm_pretrained:.4f}',
        f'{f1_lstm_scratch:.4f}'
    ],
    'Test Loss': [
        loss_tfidf,
        f'{loss_w2v:.4f}',
        f'{loss_lstm_pretrained:.4f}',
        f'{loss_lstm_scratch:.4f}'
    ]
})

print("\nBẢNG SO SÁNH KẾT QUẢ:\n")
print(results_df.to_string(index=False))
print("\n")

# Xác định mô hình tốt nhất
f1_scores = [f1_tfidf, f1_w2v, f1_lstm_pretrained, f1_lstm_scratch]
best_model_idx = np.argmax(f1_scores)
model_names = ['TF-IDF + LR', 'Word2Vec + Dense', 'LSTM Pre-trained', 'LSTM Scratch']

print(f"Mô hình tốt nhất: {model_names[best_model_idx]} với F1-score: {f1_scores[best_model_idx]:.4f}")
print("\n")

PHẦN 1: SO SÁNH ĐỊNH LƯỢNG CÁC MÔ HÌNH
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step

BẢNG SO SÁNH KẾT QUẢ:

                      Pipeline F1-score (Macro) Test Loss
  TF-IDF + Logistic Regression           0.8353       N/A
        Word2Vec (Avg) + Dense           0.8202    0.7011
Embedding (Pre-trained) + LSTM           0.8420    0.5182
    Embedding (Scratch) + LSTM           0.7944    0.8450


Mô hình tốt nhất: LSTM Pre-trained với F1-score: 0.8420




In [72]:
print("PHẦN 2: PHÂN TÍCH ĐỊNH TÍNH - KIỂM TRA CÂU PHỨC TẠP")

# Các câu test phức tạp + nhãn thật
test_sentences = [
    ("can you remind me to not call my mom", "reminder_create"),
    ("is it going to be sunny or rainy tomorrow", "weather_query"),
    ("find a flight from new york to london but not through paris", "flight_search")
]

def predict_sentence(text, models_dict):
    results = {}

    # Model 1: TF-IDF + LR
    pred_tfidf = models_dict['tfidf'].predict([text])[0]
    pred_tfidf_label = label_encoder.inverse_transform([pred_tfidf])[0]
    results['TF-IDF + LR'] = pred_tfidf_label
    
    # Model 2: Word2Vec + Dense
    vec = sentence_to_avg_vector(text, w2v_model).reshape(1, -1)
    pred_w2v = np.argmax(models_dict['w2v'].predict(vec, verbose=0), axis=1)[0]
    pred_w2v_label = label_encoder.inverse_transform([pred_w2v])[0]
    results['Word2Vec + Dense'] = pred_w2v_label
    
    # Model 3: LSTM Pre-trained
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred_lstm_pre = np.argmax(models_dict['lstm_pre'].predict(padded, verbose=0), axis=1)[0]
    pred_lstm_pre_label = label_encoder.inverse_transform([pred_lstm_pre])[0]
    results['LSTM Pre-trained'] = pred_lstm_pre_label
    
    # Model 4: LSTM Scratch
    pred_lstm_scratch = np.argmax(models_dict['lstm_scratch'].predict(padded, verbose=0), axis=1)[0]
    pred_lstm_scratch_label = label_encoder.inverse_transform([pred_lstm_scratch])[0]
    results['LSTM Scratch'] = pred_lstm_scratch_label
    
    return results

# Dictionary chứa các mô hình
models_dict = {
    'tfidf': tfidf_lr_pipeline,
    'w2v': model_w2v,
    'lstm_pre': lstm_model_pretrained,
    'lstm_scratch': lstm_model_scratch
}

# Phân tích từng câu
for idx, (sentence, true_label) in enumerate(test_sentences, 1):
    print(f"{'='*50}")
    print(f"CÂU {idx}: \"{sentence}\"")
    print(f"Ý ĐỊNH ĐÚNG: {true_label}")
    print(f"{'='*50}")
    
    predictions = predict_sentence(sentence, models_dict)
    
    print("\nDự đoán từ các mô hình:")
    correct_models = []
    
    for model_name, intent in predictions.items():
        status = "ĐÚNG" if intent == true_label else "SAI"
        print(f"- {model_name}: {intent}   --> {status}")
        if intent == true_label:
            correct_models.append(model_name)

    # Tự động nhận xét
    print("\nNHẬN XÉT:")
    if len(correct_models) == 0:
        print("Không mô hình nào dự đoán đúng câu này.")
    else:
        print(f"Mô hình dự đoán đúng: {', '.join(correct_models)}")
    
    print("\n")


PHẦN 2: PHÂN TÍCH ĐỊNH TÍNH - KIỂM TRA CÂU PHỨC TẠP
CÂU 1: "can you remind me to not call my mom"
Ý ĐỊNH ĐÚNG: reminder_create

Dự đoán từ các mô hình:
- TF-IDF + LR: calendar_set   --> SAI
- Word2Vec + Dense: email_sendemail   --> SAI
- LSTM Pre-trained: calendar_set   --> SAI
- LSTM Scratch: calendar_set   --> SAI

NHẬN XÉT:
Không mô hình nào dự đoán đúng câu này.


CÂU 2: "is it going to be sunny or rainy tomorrow"
Ý ĐỊNH ĐÚNG: weather_query

Dự đoán từ các mô hình:
- TF-IDF + LR: weather_query   --> ĐÚNG
- Word2Vec + Dense: weather_query   --> ĐÚNG
- LSTM Pre-trained: weather_query   --> ĐÚNG
- LSTM Scratch: weather_query   --> ĐÚNG

NHẬN XÉT:
Mô hình dự đoán đúng: TF-IDF + LR, Word2Vec + Dense, LSTM Pre-trained, LSTM Scratch


CÂU 3: "find a flight from new york to london but not through paris"
Ý ĐỊNH ĐÚNG: flight_search

Dự đoán từ các mô hình:
- TF-IDF + LR: general_negate   --> SAI
- Word2Vec + Dense: transport_query   --> SAI
- LSTM Pre-trained: transport_query   --> SAI
- LST